In [1]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors

#FP used in this system uses different ranges for each radius to reduce bit collision. ie 0~2047 for radius 0, 2048~4095 for radius 1, and 4096~6143 for radius 2
def FP_generator(SMILES,radi): # radi is radius for morgan fingerprint
    binary = np.zeros((2048*(radi+1)), int)
    mol = Chem.MolFromSmiles(SMILES)
    mol_H = Chem.AddHs(mol)
    mol_bi_H = {}
    for r in range(radi+1):
        mol_fp_H = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol_H, radius=r, bitInfo=mol_bi_H, nBits = 2048)
        mol_bi_H_QC = []
        for i in mol_fp_H.GetOnBits():
            idx = mol_bi_H[i][0][0]
            radius_list = []
            for j in range(len(mol_bi_H[i])):
                atom_radi = mol_bi_H[i][j][1]
                radius_list.append(atom_radi) 
            atom = mol_H.GetAtomWithIdx(idx)
            symbol = atom.GetSymbol()
            neigbor = [x.GetAtomicNum() for x in atom.GetNeighbors()]
            if r in radius_list: #and symbol == 'C' and 1 in neigbor:#radius = 2, atom = Carbon, H possessed Carbon
                mol_bi_H_QC.append(i)
        bits = mol_bi_H_QC
        for i in bits:
            binary[(2048*r)+i] = 1
    return binary

In [3]:
import pickle5, numpy as np
with open("../my_data/data/hsqc_ms_pairs/train.pkl", "rb") as f:
    my_obj = pickle5.load(f)
print(type(my_obj))

<class 'dict'>


In [9]:
sm, fp = zip(*[(v["SMILES"], v["FP"])for v in my_obj.values()])

In [10]:
fp0, sm0 = fp[0], sm[0]
fp_hyun = FP_generator(sm0, 2)